<a href="https://colab.research.google.com/github/nicchic/NLP/blob/main/mental_t5_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large")


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("Amod/mental_health_counseling_conversations")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    context_tokens = tokenizer(examples['Context'], padding="max_length", truncation=True, max_length=128)
    response_tokens = tokenizer(examples['Response'], padding="max_length", truncation=True, max_length=128)

    return {
        "context_input_ids": context_tokens["input_ids"],
        "context_attention_mask": context_tokens["attention_mask"],
        "response_input_ids": response_tokens["input_ids"],
        "response_attention_mask": response_tokens["attention_mask"]
    }


In [ ]:
tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
attention_mask = torch.tensor(tokenized_ds['train']['context_attention_mask'][0]).unsqueeze(0).to(device)


In [ ]:
input_ids = torch.tensor(tokenized_ds['train']['context_input_ids'][0]).unsqueeze(0).to(device)


In [ ]:
output_beam = model.generate(input_ids,
                        max_length=300,
                        min_length=100,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        num_beams= 10,
                        do_sample= False,
                        no_repeat_ngram_size = 2
                        )

generated_text_beam = tokenizer.decode(output_beam[0], skip_special_tokens=True)

print("\nGenerated Text (Beam Search):")
print(generated_text_beam)


Generated Text (Beam Search):
I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone? I can't think of a specific way to do this. I think it's best to start by focusing on yourself. If you want to change your feelings of worthlessness, you'll need to work on your self-esteem. You're worth more than anyone else in this world, and you deserve to be here.


In [ ]:
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    max_length=300,
    min_length=100,
    do_sample=True,
    top_p=0.9,
)

generated_text_top = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text (Top-p Sampling):")
print(generated_text_top)


Generated Text (Top-p Sampling):
Let yourself know that you are not worthless to anyone. Try to believe that you're worth something and try to accept yourself for who you are. Give yourself positive feedback and tell yourself a lot of positive things about yourself. Then tell yourself that you are worth something. Take some time each day to reflect on how you feel and think about what you are worthy of. Be aware that this may take a little while. The sooner you start taking good care of yourself, the more easily you'll feel worth being here.


In [ ]:
pip install rouge_score

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load
rouge = load("rouge")


In [ ]:
reference_text = "Here are few things that you could do to make a positive change of you feel about yourself. Take baby steps towards your goals but take these steps everyday. List all the things that you want to do in life and then start working on one thing at a time. Give yourself credit for even the smallest accomplishment and don't forget to celebrate the fact that you've tried. always believe in yourself, you are worth everything to your loved ones"



In [ ]:
rouge_result_top_p = rouge.compute(predictions=[generated_text_top], references=[reference_text])

print("\nROUGE Scores (Top-p Sampling):")

def display_rouge_scores(rouge_scores):
    for rouge_type, score in rouge_scores.items():
        print(f"{rouge_type.upper()}: {score:.4f}")

display_rouge_scores(rouge_result_top_p)


ROUGE Scores (Top-p Sampling):
ROUGE1: 0.4407
ROUGE2: 0.0914
ROUGEL: 0.2034
ROUGELSUM: 0.2034


In [ ]:
rouge_result_beam = rouge.compute(predictions=[generated_text_beam], references=[reference_text])
print("\nROUGE Scores (Beam Search):")
display_rouge_scores(rouge_result_beam)


ROUGE Scores (Beam Search):
ROUGE1: 0.3171
ROUGE2: 0.0370
ROUGEL: 0.1585
ROUGELSUM: 0.1585
